# Student Admission - MRSort and NCS
Ariane Dalens  
Lucas Sor  
Magali Morin  

## Générateur de dataset

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(123)


def generateur(N, m):
    """
    Crée un jeu de données de taille N sur la base de 12 notes 

    args:
    N : taille du jeu de données 
    m : nombre de notes

    output: 
    df : jeu de données 
    betas : frontière
    weights : weights du modèle
    lbd : lambda du modèle
    """

    # Générer la frontière betas
    betas = rng.integers(low=8, high=15, size=m)

    # Générer les weights et les normaliser
    weights = rng.random(m)
    weights = weights / weights.sum()

    # Générer lambda
    lbd = rng.random()

    # Générer df - sigmas: notes des students
    sigmas = rng.integers(low=0, high=21, size=(N, m))
    df = pd.DataFrame(sigmas)
    df['accepted'] = df.apply(
        lambda x: classifier(x, betas, weights, lbd), axis=1)

    return df, betas, weights, lbd


def classifier(sigma, betas, weights, lbd):
    """
    Fonction de classification pour un élève alpha 

    args:
    alpha : notes d'un élève
    betas : frontière
    weights : weights du modèle
    lbd : lambda du modèle

    output: 
    1 si l'élève est accepté
    0 sinon 
    """
    return ((sigma >= betas)*weights).sum() >= lbd

def analyse_dataset(df):
    """
    Fonction d'analyse d'un dataset

    args:
    df : jeu de données 
    """
    print('---------Analyse----------')
    print(f"Nombre d'élèves acceptés: {df.accepted.sum()}")
    print("% d'élèves acceptés {:.2f} %".format(df.accepted.sum()/len(df)))
    #plt.hist(df[0])
    pass

df, betas, weights, lbd = generateur(150, 10)
analyse_dataset(df)
df

---------Analyse----------
Nombre d'élèves acceptés: 40
% d'élèves acceptés 0.27 %


,0,1,2,3,4,5,6,7,8,9,accepted
0,9,19,15,4,17,16,4,10,16,4,True
1,5,3,0,10,0,12,8,3,7,0,False
2,3,9,1,15,5,19,3,13,10,19,False
3,12,18,9,4,3,18,4,15,11,5,False
4,8,16,9,18,19,6,11,11,11,1,True
...,...,...,...,...,...,...,...,...,...,...,...
145,4,0,11,17,0,3,18,12,16,12,False
146,19,4,17,9,0,8,8,10,3,17,False
147,6,19,20,1,15,9,10,6,8,16,False
148,12,7,14,10,15,14,18,13,15,15,True


## Programme linéaire (to be implemented)

In [ ]:
from gurobipy import *
import numpy as np

# Instanciation du modèle
m = Model("MR-Sort")

# Parameters
EPSILON = 1e-6
M = 1e2

nb_students = 100
nb_grades = 3  

df, betas, weights, lbd = generateur(nb_students, nb_grades)

# Création d'un vecteur de 2 variables continues
sigmas_ = m.addMVar(shape=nb_students)
betas_ = m.addMVar(shape=nb_grades)
weights_ = m.addMVar(shape=nb_grades)
lambda_ = m.addMVar()

# ajout des contraintes 
# Sur A 

# Sur R 

m.addConstr(sigmas_ <= 1)

# maj du modèle
m.update()

# Paramétrage (mode mute)
#m.params.outputflag = 0

m.setObjective(sigmas_.sum(), GRB.MAXIMIZE)

# Résolution du PL
m.optimize()

# Affichage en mode texte du PL
print(m.display())

In [2]:
from gurobipy import *
import numpy as np

# Instanciation du modèle
m = Model("PL modelling using matrix")

# Création d'un vecteur de 2 variables continues
v = m.addMVar(shape=2)

# maj du modèle
m.update()

# Contraintes
A = np.array([ [1,    -1],
               [1,     1],
               [-0.25, 1] ])
b = np.array([4, 4, 1])
m.addConstr(A @ v <= b)

# Fonction Objectif
obj = np.array([0.0, 1.0])
m.setObjective(obj @ v, GRB.MAXIMIZE)

# Paramétrage (mode mute)
m.params.outputflag = 0

# Résolution du PL
m.optimize()

# Affichage en mode texte du PL
print(m.display())

print("La solution optimale est (v0, v1) = {} avec pour objectif z = {}".format((v.X), m.objVal))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-30
None
La solution optimale est (v0, v1) = [2.4 1.6] avec pour objectif z = 1.6


## Solver SAT - gophersat

Le solveur **gophersat** est un programme open-source écrit en [Go](https://golang.org/). Ses sources sont accessibles sur [https://github.com/crillab/gophersat](https://github.com/crillab/gophersat). Une version compilée est disponible sur le EDUNAO du cours. C'est un solveur SAT, MAX-SAT et pseudo-booléen. Il est également capable de faire du comptage de modèles.

In [ ]:
#Construction du DIMCS et Résolution

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], {i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 
